In [3]:
from astroquery.gaia import Gaia
from astroquery.vizier import Vizier
import astropy.units as u
import pandas as pd

def cross_reference_gaia_starhorse(ra, dec, radius_deg):
    # Perform cone search on Gaia EDR3
    gaia_query = f"SELECT * FROM gaiadr2.gaia_source WHERE CONTAINS(POINT('ICRS', gaiadr2.gaia_source.ra, gaiadr2.gaia_source.dec), CIRCLE('ICRS', {ra}, {dec}, {radius_deg})) = 1"
    gaia_job = Gaia.launch_job(gaia_query)
    gaia_result = gaia_job.get_results()

    # Perform cone search on StarHorse using VizieR
    vizier = Vizier(columns=["*", "_RAJ2000", "_DEJ2000"], row_limit=-1)
    starhorse_query_result = vizier.query_region(
        coordinates=f"{ra} {dec}",
        radius=radius_deg * u.deg,
        catalog="J/A+A/649/A6"
    )

    # Check if the result is non-empty
    if starhorse_query_result:
        starhorse_result = starhorse_query_result[0]
    else:
        starhorse_result = None

    return gaia_result, starhorse_result

def save_to_csv(results, filename):
    if results is not None:
        # Convert the Astropy Table to a Pandas DataFrame
        df = results.to_pandas()

        # Save the DataFrame to a CSV file
        df.to_csv(filename, index=False)
    else:
        print("Results are empty. Not saving to CSV.")

def main():
    # Coordinates for the cone search (RA, Dec in degrees)
    ra = 6.41
    dec = 9.53
    radius_deg = 0.3

    # Perform cross-referencing with cone search
    gaia_result, starhorse_result = cross_reference_gaia_starhorse(ra, dec, radius_deg)

    # Save the results to a single CSV file
    save_to_csv(gaia_result, 'gaia_results.csv')
    save_to_csv(starhorse_result, 'starhorse_results.csv')

    # Display the results
    print("Result from Gaia EDR3:")
    print(gaia_result)
    print("\nResult from StarHorse:")
    print(starhorse_result)

if __name__ == "__main__":
    main()


SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1002)